# Basic ML Model Deployment

## Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import StandardScaler
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

## Fetch Data

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/tkseneee/Dataset/master/Loan_data_ver2.csv')
#data=pd.read_csv('Loan_data_ver2.csv')

## Explore Data

In [3]:
data.shape

(614, 6)

In [4]:
data.dtypes

Married             object
Education           object
ApplicantIncome      int64
LoanAmount         float64
Credit_History     float64
Loan_Status        float64
dtype: object

In [5]:
data.head(2)

,Married,Education,ApplicantIncome,LoanAmount,Credit_History,Loan_Status
0,No,Graduate,5849,NaN,1.0,0.10
1,Yes,Graduate,4583,128.0,1.0,0.32


In [6]:
# fetch features with missing values
data.isnull().sum()

Married             3
Education           0
ApplicantIncome     0
LoanAmount         22
Credit_History     50
Loan_Status         0
dtype: int64

3 features namely - Married,LoanAmount,Credit_History has missing values

In [7]:
data['Married'].value_counts()

Married
Yes    398
No     213
Name: count, dtype: int64

In [8]:
data['Education'].value_counts()

Education
Graduate        449
Not Graduate    127
HSC              38
Name: count, dtype: int64

In [9]:
# segreegating target & feature
X=data.drop('Loan_Status', axis=1)
y=data['Loan_Status']

In [10]:
# spliting data into train & validation set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=48)

In [11]:
# fetching numeric features list
feat_num=list(X.select_dtypes(include=np.number).columns)


In [12]:
# fetching categorical features  list
feat_cat=list(X.select_dtypes(exclude=np.number).columns)

In [13]:
feat_cat

['Married', 'Education']

## Defining Data processing & Modeling  Pipeline

In [14]:
#  pipeline for numeric atures -missing values replacement using k-Nearest Neighbors follwed by StandardScaler() 
num_pipe=Pipeline([('imputer',KNNImputer()),('std_scale',StandardScaler())])



In [15]:
# pipeline for categorical faetures - missing category replacement by new category i.e. missing followed by one hot encoding 
feat_pipe = Pipeline([('imputer',SimpleImputer(strategy='constant', fill_value='Missing')), 
                      ('one_hot',(OneHotEncoder()))]) 



In [16]:
#combine data processing pipeline
data_pipeline=ColumnTransformer([('numeric',num_pipe,feat_num),
                                 ('categorical',feat_pipe, feat_cat)],
                                remainder='passthrough')



In [17]:
data_pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('std_scale',
                                                  StandardScaler())]),
                                 ['ApplicantIncome', 'LoanAmount',
                                  'Credit_History']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Missing',
                                                                strategy='constant')),
                                                 ('one_hot', OneHotEncoder())]),
                                 ['Married', 'Education'])])

In [18]:
# adding ml-model into pipeline 
full_pipe=Pipeline([('pre_process',data_pipeline),('model',RandomForestRegressor())])

In [19]:
# training
full_pipe.fit(X_train,y_train)

Pipeline(steps=[('pre_process',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('std_scale',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'LoanAmount',
                                                   'Credit_History']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder())]),
                                                  ['Married', 'Education'])])),
                ('model', RandomForestRegressor())])

In [20]:
# prediction
full_pipe.predict(X_test)

array([0.1653, 0.2097, 0.5636, 0.3916, 0.1858, 0.1004, 0.9588, 0.2359,
       0.2611, 0.286 , 0.1916, 0.9751, 0.2706, 0.1539, 0.0576, 0.0863,
       0.1141, 0.0449, 0.0575, 0.2534, 0.3111, 0.0754, 0.3323, 0.4691,
       0.5947, 0.1608, 0.2446, 0.2567, 0.1995, 0.0148, 0.0903, 0.2474,
       0.425 , 0.9533, 0.1057, 0.2836, 0.2002, 0.6127, 0.9458, 0.204 ,
       0.2805, 0.135 , 0.2785, 0.3643, 0.3993, 0.5079, 0.1186, 0.1819,
       0.2486, 0.1699, 0.0935, 0.4085, 0.0591, 0.0864, 0.9637, 0.0066,
       0.4427, 0.3259, 0.2711, 0.2173, 0.3195, 0.8336, 0.2746, 0.1244,
       0.459 , 0.2415, 0.0688, 0.2717, 0.1653, 0.2037, 0.378 , 0.1429,
       0.2969, 0.3018, 0.082 , 0.8358, 0.1168, 0.4706, 0.5805, 0.0633,
       0.299 , 0.1278, 0.3246, 0.7851, 0.4403, 0.1571, 0.1486, 0.1094,
       0.0538, 0.4851, 0.0501, 0.2159, 0.2385, 0.5309, 0.4227, 0.1828,
       0.9716, 0.0746, 0.0799, 0.3873, 0.4836, 0.0546, 0.0054, 0.0416,
       0.0941, 0.0927, 0.3246, 0.2197, 0.049 , 0.1169, 0.164 , 0.1191,
      

In [21]:
## can store numeric and categorical variables also as pickle file
pickle.dump(feat_num,open('feat_numv1','wb'))
pickle.dump(feat_cat,open('feat_catv1','wb'))

 

## Store the model as pickle file 

In [22]:
pickle.dump(full_pipe,open('full_pipeline','wb'))